In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import glob
import logging
import os
from PIL import Image

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# 1读取文件夹

In [ ]:
imgs_path=glob.glob('Desktop/DP/fruit/*/*.jpg')


In [ ]:
len(imgs_path)

1200

In [ ]:
img_p=imgs_path[5]
img_p

'Desktop/DP/fruit\\FreshApple\\FreshApple (103).jpg'

In [ ]:
img_p.split("\\")[1]

'FreshApple'

In [ ]:
all_labels_name=[img_p.split("\\")[1] for img_p in imgs_path]

In [ ]:
all_labels_name[:6]

['FreshApple',
 'FreshApple',
 'FreshApple',
 'FreshApple',
 'FreshApple',
 'FreshApple']

In [ ]:
label_name = np.unique(all_labels_name)
len(label_name)

6

In [ ]:
label_to_index=dict((name,i)for i,name in enumerate(label_name))
label_to_index

{'FreshApple': 0,
 'FreshBanana': 1,
 'FreshOrange': 2,
 'RottenApple': 3,
 'RottenBanana': 4,
 'RottenOrange': 5}

In [ ]:
index_to_label = dict((v,k)for k,v in label_to_index.items())
index_to_label

{0: 'FreshApple',
 1: 'FreshBanana',
 2: 'FreshOrange',
 3: 'RottenApple',
 4: 'RottenBanana',
 5: 'RottenOrange'}

In [ ]:
all_labels=[label_to_index.get(name)for name in all_labels_name]

# 数据集

In [ ]:
random_index = np.random.permutation(len(imgs_path))

In [ ]:
imgs_path=np.array(imgs_path)[random_index]
all_labels=np.array(all_labels)[random_index]

In [ ]:
i=int(len(imgs_path)*0.8)

In [ ]:
train_path=imgs_path[:i]
train_labels=all_labels[:i]
test_path=imgs_path[i:]
test_labels=all_labels[i:]

In [ ]:
train_ds=tf.data.Dataset.from_tensor_slices((train_path,train_labels))
test_ds=tf.data.Dataset.from_tensor_slices((test_path,test_labels))

In [ ]:
def load_img(path,label):
  image = tf.io.read_file(path)
  image = tf.image.decode_jpeg(image,channels=3)
  image = tf.image.resize(image,[256,256])
  image = tf.cast(image,tf.float32)
  image = image/255
  return image,label

In [ ]:
train_ds = train_ds.map(load_img)
test_ds = test_ds.map(load_img)
train_ds

<MapDataset element_spec=(TensorSpec(shape=(256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [ ]:
BATCH_SIZE=32
train_ds = train_ds.repeat().shuffle(300).batch(BATCH_SIZE)
train_ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
test_ds = test_ds.batch(BATCH_SIZE)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64,(3,3),input_shape=(256,256,3),
                           activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256,(3,3),activation='relu'),
    tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.MaxPool2D(),
    # tf.keras.layers.Conv2D(512,(3,3),activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(6)
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 64)      1792      
                                                                 
 batch_normalization (BatchN  (None, 254, 254, 64)     256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 252, 252, 64)      36928     
                                                                 
 batch_normalization_1 (Batc  (None, 252, 252, 64)     256       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 126, 126, 64)     0         
 )                                                               
                                                        

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['acc']
              )

In [ ]:
train_count = len(train_path)
test_count = len(test_path)

In [ ]:
steps_per_epoch = train_count//BATCH_SIZE
validation_steps = test_count//BATCH_SIZE

In [ ]:
history = model.fit(train_ds,epochs=5,
      steps_per_epoch = steps_per_epoch,
      validation_data = test_ds,
      validation_steps=validation_steps)

Epoch 1/5
30/30 [==============================] - 20s 598ms/step - loss: 0.0246 - acc: 0.9948 - val_loss: 0.3838 - val_acc: 0.8705
Epoch 2/5
30/30 [==============================] - 17s 585ms/step - loss: 0.0307 - acc: 0.9906 - val_loss: 0.3840 - val_acc: 0.8571
Epoch 3/5
30/30 [==============================] - 18s 592ms/step - loss: 0.0423 - acc: 0.9875 - val_loss: 0.3557 - val_acc: 0.9062
Epoch 4/5
30/30 [==============================] - 18s 605ms/step - loss: 0.0353 - acc: 0.9906 - val_loss: 0.4995 - val_acc: 0.8170
Epoch 5/5
30/30 [==============================] - 18s 597ms/step - loss: 0.0217 - acc: 0.9948 - val_loss: 0.0827 - val_acc: 0.9732


In [ ]:
history.history.keys()

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [ ]:
plt.plot(history.epoch, history.history.get('acc'), label='acc')
plt.plot(history.epoch, history.history.get('val_acc'), label='val_acc')
plt.legend()

In [ ]:
plt.plot(history.epoch, history.history.get('loss'), label='loss')
plt.plot(history.epoch, history.history.get('val_loss'), label='val_loss')
plt.legend()

In [ ]:
def load_and_preprocess_image(path):
  image = tf.io.read_file(path)
  image = tf.io.decode_jpeg(image,channels=3)
  image = tf.image.resize(image,[256,256])
  image = tf.cast(image,tf.float32)
  image = image/255.0
  return image

In [ ]:
test_img = 'Downloads/fruit111/Augmented Image/Augmented Image/FreshBanana/FreshBanana (6).jpg'
test_tensor = load_and_preprocess_image(test_img)
test_tensor = tf.expand_dims(test_tensor,axis=0)
pred = model.predict(test_tensor)

1/1 [==============================] - 0s 24ms/step


In [ ]:
index_to_label.get(np.argmax(pred))

'FreshBanana'

In [ ]:
!mkdir -p saved_model
model.save('Desktop/DP/newmodel/saved_model/my_model')

子目录或文件 -p 已经存在。
处理: -p 时出错。
子目录或文件 saved_model 已经存在。
处理: saved_model 时出错。


INFO:tensorflow:Assets written to: Desktop/DP/newmodel/saved_model/my_model\assets


INFO:tensorflow:Assets written to: Desktop/DP/newmodel/saved_model/my_model\assets


保存模型

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('Desktop/DP/newmodel/saved_model/my_model') # path to the SavedModel directory
tflite_model = converter.convert()

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()
tflite_model_quant_file.write_bytes(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Chace\AppData\Local\Temp\tmp5dau2_ug\assets


INFO:tensorflow:Assets written to: C:\Users\Chace\AppData\Local\Temp\tmp5dau2_ug\assets


2719192

In [ ]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

# 量化

In [ ]:
import pathlib

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model_quant = converter.convert()
tflite_model_quant_file.write_bytes(tflite_model_quant)

INFO:tensorflow:Assets written to: C:\Users\Chace\AppData\Local\Temp\tmpdn7zkp7j\assets


INFO:tensorflow:Assets written to: C:\Users\Chace\AppData\Local\Temp\tmpdn7zkp7j\assets


698992

In [ ]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model_quant)